# FlascDataFrame

FLASC has historically used a `pandas.DataFrame` to store the data to be processed.  Beginning in version 2.1, the `FlascDataFrame` class was introduced to provide additional methods and functionality to the data.  `FlascDataFrame` is a subclass of `pandas.DataFrame` and can be used in place of a `pandas.DataFrame`.  This notebook provides an overview of the `FlascDataFrame` class and its methods.  Support is added for converting between "flasc" style data formatting and "user" formats

## FlascFormat

FlascFormat of a dataframe where columns appear according to a certain convention:

- `time` represents the time, preferably in UTC
-  turbines are sequentially numbered, starting from 0, numbers are always 3 digits long
-  `pow_000` represents the power output of turbine 0
-  `ws_000` represents the wind speed at turbine 0
-  `wd_000` represents the wind direction at turbine 0
-  `wd` represents the wind direction chosen for example to represent the overall inflow direction
-  `ws` represents the wind speed chosen for example to represent the overall inflow speed
-  `pow_ref` represents the power output of the reference turbine (or average of reference turbines)
-  `pow_test` represents the power output of the test turbine (or average of test turbines)

In [22]:
# A dataframe to used in flasc initialized as a normal pandas dataframe
import pandas as pd

# This dataframe could be used for flasc functions
df = pd.DataFrame(
    {
        "time": [0, 1, 2, 3, 4, 5],
        "pow_000": [100, 100, 100, 100, 100, 100],
        "pow_001": [100, 100, 100, 100, 100, 100],
        "ws_000": [10, 10, 10, 10, 10, 10],
        "ws_001": [10, 10, 10, 10, 10, 10],
        "wd_000": [270, 270, 270, 270, 270, 270],
        "wd_001": [270, 270, 270, 270, 270, 270],
    }
)

## Using FlascDataFrame

In [23]:
# The above dataframe could be converted to a FlascDataFrame directly
from flasc import FlascDataFrame

fdf = FlascDataFrame(df)
print(fdf.head())

FlascDataFrame in FLASC format
   time  pow_000  pow_001  ws_000  ws_001  wd_000  wd_001
0     0      100      100      10      10     270     270
1     1      100      100      10      10     270     270
2     2      100      100      10      10     270     270
3     3      100      100      10      10     270     270
4     4      100      100      10      10     270     270


In [24]:
# The FlascDataFrame includes a few helper functions added to the base pandas dataframe
fdf.n_turbines

2

## Creating a FlascDataFrame from User Data

More value from a FlascDataFrame is obtained when using it convert back and forth between user-formatted data and Flasc Data.  

In [25]:
import numpy as np

# Suppose the we have a 3 turbine farm with turbines names 'TB01', 'TB02', 'TB03'
# For each turbine we have power, wind speed and wind direction data
# Assume that in the native data collection system,
# the signal names for each channel are given below

N = 20  # Number of data points

# Wind speeds
wind_speed_TB01 = np.random.rand(N) + 8.0
wind_speed_TB02 = np.random.rand(N) + 7.5
wind_speed_TB03 = np.random.rand(N) + 8.5

# Wind directions
wind_dir_TB01 = 10 * np.random.rand(N) + 270.0
wind_dir_TB02 = 10 * np.random.rand(N) + 270.0
wind_dir_TB03 = 10 * np.random.rand(N) + 270.0

# Power
power_TB01 = wind_speed_TB01**3
power_TB02 = wind_speed_TB02**3
power_TB03 = wind_speed_TB03**3

# Time
time = np.arange(N)

In [39]:
# Add this data to a pandas dataframe
df = pd.DataFrame(
    {
        "time": time,
        "wind_speed_TB01": wind_speed_TB01,
        "wind_speed_TB02": wind_speed_TB02,
        "wind_speed_TB03": wind_speed_TB03,
        "wind_dir_TB01": wind_dir_TB01,
        "wind_dir_TB02": wind_dir_TB02,
        "wind_dir_TB03": wind_dir_TB03,
        "power_TB01": power_TB01,
        "power_TB02": power_TB02,
        "power_TB03": power_TB03,
    }
)

The data is currently stored using the the channel and turbine names of the user, by supplying additional metadata to the FlascDataFrame, the data can be converted to and from the FlascFormat.  

In [40]:
# Declare a name_map dictionary to map the signal names to the turbine names.
# The turbine numbers when 0-indexed in FLASC format should
# align with their numbering in the FLORIS model of the same farm.
channel_name_map = {
    "time": "time",
    "wind_speed_TB01": "ws_000",
    "wind_speed_TB02": "ws_001",
    "wind_speed_TB03": "ws_002",
    "wind_dir_TB01": "wd_000",
    "wind_dir_TB02": "wd_001",
    "wind_dir_TB03": "wd_002",
    "power_TB01": "pow_000",
    "power_TB02": "pow_001",
    "power_TB03": "pow_002",
}

In [43]:
## Declare an instance of FlascDataFrame
fdf = FlascDataFrame(df, channel_name_map=channel_name_map)
print(fdf.head())

FlascDataFrame in user (wide) format
   time  wind_speed_TB01  wind_speed_TB02  wind_speed_TB03  wind_dir_TB01  \
0     0         8.112750         8.402828         8.596124     272.557566   
1     1         8.276103         7.651271         9.360673     274.498764   
2     2         8.767538         7.900008         9.028346     271.337235   
3     3         8.276872         8.208943         8.911937     277.227258   
4     4         8.828375         8.039787         8.925044     278.484343   

   wind_dir_TB02  wind_dir_TB03  power_TB01  power_TB02  power_TB03  
0     271.263082     276.458594  533.954502  593.302928  635.196362  
1     279.425298     272.171584  566.862317  447.920293  820.202867  
2     275.176022     273.955338  673.958144  493.040519  735.909803  
3     277.600801     279.706961  567.020466  553.173872  707.809508  
4     270.516648     277.232803  688.085250  519.677149  710.936856  


/Users/pfleming/Projects/FLASC/flasc/flasc/flasc_dataframe.py:57: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.channel_name_map = channel_name_map


In [45]:
# Convert now into flasc format
fdf_flasc = fdf.convert_to_flasc_format()
print(fdf_flasc.head())

FlascDataFrame in FLASC format
   time    ws_000    ws_001    ws_002      wd_000      wd_001      wd_002  \
0     0  8.112750  8.402828  8.596124  272.557566  271.263082  276.458594   
1     1  8.276103  7.651271  9.360673  274.498764  279.425298  272.171584   
2     2  8.767538  7.900008  9.028346  271.337235  275.176022  273.955338   
3     3  8.276872  8.208943  8.911937  277.227258  277.600801  279.706961   
4     4  8.828375  8.039787  8.925044  278.484343  270.516648  277.232803   

      pow_000     pow_001     pow_002  
0  533.954502  593.302928  635.196362  
1  566.862317  447.920293  820.202867  
2  673.958144  493.040519  735.909803  
3  567.020466  553.173872  707.809508  
4  688.085250  519.677149  710.936856  
